In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Nick\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# Gather latest news information from redplanetscience.com
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find all news story elements
results = soup.find_all('div', class_='list_text')

# Iterate through found story elements
for result in results[0]:
    # collect latest news title
    ltst_news_title = soup.find('div', class_='content_title').get_text()
    
    # collect latest news paragraph
    ltst_news_text = soup.find('div', class_='article_teaser_body').get_text()

# Check results
print(ltst_news_title)
print("-------")
print(ltst_news_text)

Media Get a Close-Up of NASA's Mars 2020 Rover
-------
The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch.


In [5]:
# Gather image from current featured mars image from spaceimages-mars.com
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [6]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find all news story elements
results = soup.find_all('div', class_='header')

for result in results[0]:
    img = soup.find('img', class_='headerimage')
    src = img['src']

# Store and check the results
featured_image_url = url + src
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


In [7]:
# Read all tables from 'https://galaxyfacts-mars.com/' into pandas
mars_table = pd.read_html('https://galaxyfacts-mars.com/')

# Determine number of tables grabbed
len(mars_table)

2

In [8]:
# Convert target table into a dataframe to check scrape results
mars_info_df = mars_table[0]

# grab the first row for the header
new_header = mars_info_df.iloc[0]

# take the data less the header row
mars_info_df = mars_info_df[1:]

# set the header row as the df header
mars_info_df.columns = new_header

# set index
mars_info_df = mars_info_df.set_index('Mars - Earth Comparison')
mars_info_df

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
mars_info_html = mars_info_df.to_html()

# test functionality
test = pd.read_html(mars_info_html)
test

[       Unnamed: 0_level_0               Mars              Earth
   Mars - Earth Comparison Unnamed: 1_level_1 Unnamed: 2_level_1
 0               Diameter:           6,779 km          12,742 km
 1                   Mass:    6.39 × 10^23 kg    5.97 × 10^24 kg
 2                  Moons:                  2                  1
 3      Distance from Sun:     227,943,824 km     149,598,262 km
 4         Length of Year:     687 Earth days        365.24 days
 5            Temperature:       -87 to -5 °C        -88 to 58°C]

In [10]:
# Scrape high-res mars images from https://marshemispheres.com/
url = 'https://marshemispheres.com/'
browser.visit(url)

In [11]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Find all links to images
image_links = soup.find_all('div', class_='item')

link_list = []

for image_link in image_links:
    link = image_link.find('a', class_='itemLink product-item')
    link_list.append(url + link['href'])

link_list

['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [31]:
# list to store image information
hemisphere_image_info = []

for link in link_list:
    
    #create dict to store image information
    img_dict = {}
    
    # go to the url with the high-quality image
    browser.visit(link)
    html = browser.html
    
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all image titles
    image_title = soup.find('h2', class_='title').get_text()
    title_strip = image_title.rstrip('Enhanced')
    
    # Find image url
    image_html = soup.find('img', class_='wide-image')
    image_url = url + image_html['src']
    
    # Store title and url in dict and add to hemisphere list
    img_dict["title"] = title_strip[0:len(title_strip) - 1]
    img_dict["img_url"] = image_url
    hemisphere_image_info.append(img_dict)

hemisphere_image_info

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [ ]:
# Close the browser after scraping
browser.quit()